# Sampling scenes in Switzerland
- Rasterise forest 
    - along a grid of resolution 1 km? 2.56 km/1.28km?
    - calculate % coverage in forest
    - align grid to Sentinel?
    
- Keep cells > 40% forest
- Save central coordinate to txt file

In [1]:
from rasterise import *

## 1. Rasterise shapefile, forest mask (1m resolution, EPSG 2056)

In [ ]:
# Define the input shapefile path
shapefile_path = '/Users/led/Desktop/Master_Thesis/Data/original_data/forest_mask_ch/ID164.23_Waldmaske_LV95/ForestMask_hull_LV95_20171127.shp'
# Define the output raster path
output_raster_path = '/Users/led/Desktop/test_raster.tif'
# Define the pixel size in meters
pixel_size = 1 

rasterise_shapefile(shapefile_path,output_raster_path, pixel_size)

## 1. Rasterise while reprojecting

In [ ]:
# Define the input shapefile path
shapefile_path = '/Users/led/Desktop/Master_Thesis/Data/original_data/forest_mask_ch/ID164.23_Waldmaske_LV95/ForestMask_hull_LV95_20171127.shp'
# Define the output raster path
output_raster_path = '/Users/led/Desktop/sampling_rasters/test_raster_reproj.tif'
# Define the pixel size in meters
pixel_size = 1 # TO PUT IN DEGREES EPSG 4326
# Define the target CRS in Well-Known Text (WKT) format
target_crs = 'EPSG:4326'

rasterise_reproject_shapefile(shapefile_path,output_raster_path, pixel_size, target_crs)

## 2. Upsample (continuous value)

In [ ]:
# Define the input raster path at 1m resolution
input_raster_path = '/Users/led/Desktop/test_raster.tif'
# Define the output raster path
output_raster_path = '/Users/led/Desktop/test_raster_20.tif'
# Define the desired resolution in meters
new_pixel_size = 20

upsample_sum_raster(input_raster_path, output_raster_path, new_pixel_size)

## 3. Reproject + normalise

In [ ]:
# Define the input raster path
input_raster_path = '/Users/led/Desktop/sampling_rasters/test_raster.tif' #'/Users/led/Desktop/test_raster_256.tif' #'/Users/led/Desktop/test_raster_20.tif'
# Define the output raster path
output_raster_path = '/Users/led/Desktop/sampling_rasters/test_raster_reproj.tif'#'/Users/led/Desktop/test_raster_256_reproj.tif' #'/Users/led/Desktop/test_raster_20_reproj.tif'
# Define the target CRS as an EPSG code
target_crs = 'EPSG:4326'

In [ ]:
reproject_normalise_raster(input_raster_path, output_raster_path, target_crs)

## 2. Combine upsampling, reprojecting, normalising

In [2]:
# Define the input raster path
input_raster_path = '/Users/led/Desktop/sampling_rasters/forest_mask.tif'
# Define the output raster path
output_raster_path = '/Users/led/Desktop/forest_mask_256_reproj.tif'
# Define the target CRS as an EPSG code
target_crs = 'EPSG:4326'
# Define the desired pixel size in FINAL CRS (degrees)
new_pixel_size_x = 0.033
new_pixel_size_y = 0.02304 

# (x,y) 20m: 0.00024067, 0.00018
# 2.56km:  0.033, 0.02304

In [3]:
upsample_reproject_normalise_raster(input_raster_path, output_raster_path, target_crs, new_pixel_size_x, new_pixel_size_y)

# Rasterise kml polygons
TODO: need to align to forest mask (reprojected) bounds when rasterising

In [ ]:
import os
from osgeo import ogr, gdal
from rasterise import *

In [ ]:
input_kml_file = "/Users/led/Desktop/Master_Thesis/Data/Brun/brown_polys_2.kml"
output_raster_file = "/Users/led/Desktop/sampling_rasters/polys_brun.tif"
forest_mask = '/Users/led/Desktop/sampling_rasters/forest_mask_reproj.tif'

In [ ]:
rasterise_kml_to_existing_raster(input_kml_file, output_raster_file, forest_mask)

### Upsample to 2.56km

In [ ]:
input_raster_path = "/Users/led/Desktop/sampling_rasters/polys_brun.tif"
output_raster_path = "/Users/led/Desktop/sampling_rasters/polys_brun_256.tif"
new_pixel_size_x = 0.033
new_pixel_size_y = 0.02304

In [ ]:
from osgeo import gdal

# Define the input raster path at 1m resolution
input_raster_path = "/Users/led/Desktop/sampling_rasters/polys_brun.tif"

# Define the output raster path
output_raster_path = "/Users/led/Desktop/sampling_rasters/polys_brun_256.tif"

# Define the desired resolution in degrees (EPSG:4326)
new_pixel_size_x = 0.033
new_pixel_size_y = 0.02304

# Open the input raster
input_raster = gdal.Open(input_raster_path)

# Get the original raster's geotransform and dimensions
original_geotransform = input_raster.GetGeoTransform()
original_width = input_raster.RasterXSize
original_height = input_raster.RasterYSize

# Compute the new dimensions based on the desired resolution
new_width = int(original_width * original_geotransform[1] / new_pixel_size_x)
new_height = int(original_height * abs(original_geotransform[5]) / new_pixel_size_y)

# Create a new raster dataset with the desired resolution
driver = gdal.GetDriverByName('GTiff')
output_raster = driver.Create(
    output_raster_path,
    new_width,
    new_height,
    1,  # number of bands
    gdal.GDT_Float32  # data type of the raster (change if needed)
)

# Compute the new geotransform
new_geotransform = (
    original_geotransform[0],
    new_pixel_size_x,
    original_geotransform[2],
    original_geotransform[3],
    original_geotransform[4],
    -new_pixel_size_y
)

# Set the projection and geotransform of the new raster dataset
output_raster.SetProjection(input_raster.GetProjection())
output_raster.SetGeoTransform(new_geotransform)

# Resample the input raster to match the new resolution
gdal.ReprojectImage(
    input_raster,
    output_raster,
    input_raster.GetProjection(),
    input_raster.GetProjection(),
    gdal.GRA_Sum
)


# Get the upsampled raster band
output_band = output_raster.GetRasterBand(1)
# Read the band values as an array
output_array = output_band.ReadAsArray()

# Normalize the values of the reprojected array to a range of 0-1
min_value = output_array.min()
max_value = output_array.max()
normalized_array = (output_array - min_value) / (max_value - min_value)

# Write the normalized array to the output raster band
normalized_band = output_raster.GetRasterBand(1)
normalized_band.WriteArray(normalized_array)

    
# Close the datasets
output_raster = None
input_raster = None

# Sampling algorithm

## 1. Sampling N scenes with drought positives
Sample scene if forest >40% and ANY drought in 2.56km scene

TO DO: check if forest and drought rasters aligned

In [1]:
from sample import * 

In [2]:
forest_mask_256 = '/Users/led/Desktop/sampling_rasters/forest_mask_256_reproj.tif'
drought_labels_256 = '/Users/led/Desktop/sampling_rasters/polys_brun_256.tif'

thresh_drought = 0
thresh_forest = 0.4

# Define the output text file path
output_file_path = '/Users/led/Desktop/sampling_rasters/coords_drought.txt'

In [3]:
sample_drought(forest_mask_256, drought_labels_256, thresh_drought, thresh_forest, output_file_path)

## Sampling N non-drought scenes
- Split forest raster in 6 subregions
- For each subregion, get randomly N/6 scenes in each subregions.
- Sample scene if forest >40% in 2.56km scene, evenly across switzerland.

In [66]:
forest_mask_256 = '/Users/led/Desktop/sampling_rasters/forest_mask_256_reproj.tif'
drought_labels_256 = '/Users/led/Desktop/sampling_rasters/polys_brun_256.tif'

thresh_forest = 0.4

# Defined number of samples desired
N = 100

# Initialise the output file
output_file = '/Users/led/Desktop/sampling_rasters/coords_sample_negative.txt'

In [67]:
sample_negatives(forest_mask_256, drought_labels_256, thresh_forest, N, output_file)